In [1]:
import pandas as pd

def fixTable(allDF, singleDF, month):
    """Takes a dataframe for a single month, fixes the headers, 
       and adds a column for the current month.
       Then concatenates it to the full table for the year."""
    # make actual table headers the df header, remove original row with that info 
    singleDF.rename(columns=singleDF.iloc[0], inplace=True)
    singleDF.rename(columns={singleDF.columns[0]: "Type"}, inplace=True)
    singleDF.drop([0], axis=0, inplace=True)
    singleDF["Month"] = month
    if allDF.empty:
        allDF = singleDF
    else:
        allDF = pd.concat([allDF, singleDF], axis="rows")
    return allDF

def processSolidWasteReport(path):
    """Loads in a Solid Waste Report from a given path, processes each table for each months,
       and makes a csv file per table containing the data for each month."""
    xl = pd.ExcelFile(path)
    # see all sheet names
    sheetNames = xl.sheet_names
    print(sheetNames, len(sheetNames))
    # initialize dataframes for the full date range
    commodityRecyclingAllDF = pd.DataFrame({'' : []})
    landfilledAllDF = pd.DataFrame({'' : []})
    swmfRecycledDivertedAllDF = pd.DataFrame({'' : []})
    mulchAllDF = pd.DataFrame({'' : []})
    totalWasteStreamAllDF = pd.DataFrame({'' : []})
    # iterate over sheets
    for month in sheetNames[0:12]:
        # read a portion of a specific sheet to DataFrame
        # month: name of sheet (formatted MONTH YYYY)
        # usecols: specify list of columns to used
        # skiprows: specify list of rows to skip
        # nrows: specify number of rows to read
        commodityRecyclingDF = xl.parse(month, skiprows=range(3), nrows=9, usecols=range(11))
        commodityRecyclingAllDF = fixTable(commodityRecyclingAllDF, commodityRecyclingDF, month)
        
        landfilledDF = xl.parse(month, skiprows=range(17), nrows=6, usecols=range(3))
        landfilledAllDF = fixTable(landfilledAllDF, landfilledDF, month)
        
        swmfRecycledDivertedDF = xl.parse(month, skiprows=range(16), nrows=7, usecols=range(5,8))
        swmfRecycledDivertedAllDF = fixTable(swmfRecycledDivertedAllDF, swmfRecycledDivertedDF, month)
        
        mulchDF = xl.parse(month, skiprows=range(15), nrows=3, usecols=range(10,12))
        mulchAllDF = fixTable(mulchAllDF, mulchDF, month)
        
        totalWasteStreamDF = xl.parse(month, skiprows=range(28), nrows=6, usecols=range(3))
        totalWasteStreamAllDF = fixTable(totalWasteStreamAllDF, totalWasteStreamDF, month)
    
    commodityRecyclingAllDF.drop(columns=["DROP OFF CENTER TOTALS"], inplace=True)
    commodityRecyclingAllDF.to_csv("../data/processed/CommodityRecycling.csv", index=False)
    landfilledAllDF.to_csv("../data/processed/Landfilled.csv", index=False)
    swmfRecycledDivertedAllDF.to_csv("../data/processed/RecycledDiverted.csv", index=False)
    mulchAllDF.rename(columns={mulchAllDF.columns[1]: "Mulch"}, inplace=True)
    mulchAllDF.to_csv("../data/processed/Mulch.csv", index=False)
    totalWasteStreamAllDF.to_csv("../data/processed/TotalWasteStream.csv", index=False)
     
processSolidWasteReport("../data/solid_waste_reports_2018.xlsx")

['2018 JAN', '2018 FEB', '2018 MAR', '2018 APRIL', '2018 MAY', '2018 JUNE', '2018 JULY', '2018 AUG', '2018 SEPT', '2018 OCT', '2018 NOV', '2018 DEC', '2018 ANNUAL TOTAL', '2018 FIRST HALF', '2018 SECOND HALF'] 15
